## End to end model:Classifier

In [1]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from keras import optimizers
from keras.applications.vgg19 import preprocess_input
from keras.utils.training_utils import multi_gpu_model
from keras.utils import to_categorical
import tensorflow as tf
import cv2
import os
import random
import h5py
import numpy as np

Using TensorFlow backend.


## 1 read feature and label

In [2]:
h5_files = ['/home/tsimage/high_speed_data/end2end_feature/feature_label_500.h5',
            '/home/tsimage/high_speed_data/end2end_feature/feature_label_1000.h5',
            '/home/tsimage/high_speed_data/end2end_feature/feature_label_1500.h5',
            '/home/tsimage/high_speed_data/end2end_feature/feature_label_2000.h5']

for i, h5_file in enumerate(h5_files):
    h5f = h5py.File(h5_file,'r')
    features_h5 = h5f['features'][:]
    labels_h5 = h5f['labels'][:]
    h5f.close()
    if (i == 0):
        print("du ###",i)
        features_h5_all = features_h5
        labels_h5_all = labels_h5
        print("du", features_h5_all.shape)
        print("du", labels_h5_all.shape)
        print("du", features_h5.shape)
        print("du", labels_h5.shape)
        features_h5 = None
        labels_h5 = None
    else:
        print("du ***",i)
        print("du", features_h5_all.shape)
        print("du", labels_h5_all.shape)
        print("du", features_h5.shape)
        print("du", labels_h5.shape)
        features_h5_all = np.append(features_h5_all, features_h5, axis = 0)
        labels_h5_all = np.append(labels_h5_all, labels_h5,  axis = 0)
        features_h5 = None
        labels_h5 = None
        
print("du", features_h5_all.shape)
print("du", labels_h5_all.shape)

du ### 0
du (500, 128, 128, 512)
du (500,)
du (500, 128, 128, 512)
du (500,)
du *** 1
du (500, 128, 128, 512)
du (500,)
du (491, 128, 128, 512)
du (491,)
du *** 2
du (991, 128, 128, 512)
du (991,)
du (500, 128, 128, 512)
du (500,)
du *** 3
du (1491, 128, 128, 512)
du (1491,)
du (500, 128, 128, 512)
du (500,)
du (1991, 128, 128, 512)
du (1991,)


In [3]:
features = features_h5_all
labels = labels_h5_all.reshape([len(labels_h5_all),1])
features_h5_all = None
labels_h5_all = None

print("du", features.shape)
print("du", labels.shape)

labels = to_categorical(labels,2)
print("du", features.shape)
print("du", labels.shape)

index = [i for i in range(len(features))]  
random.shuffle(index)
features = features[index]
labels = labels[index]

print("du", features.shape)
print("du", labels.shape)

du (1991, 128, 128, 512)
du (1991, 1)
du (1991, 128, 128, 512)
du (1991, 2)
du (1991, 128, 128, 512)
du (1991, 2)


## 2 build classifier

In [4]:
with tf.device('/cpu:0'):
    input_tensor = Input((128, 128, 512))
    
    conv1_1 = Conv2D(filters = 1024, kernel_size = (3, 3), strides = (1, 1),
                   activation='relu', padding='same', name='conv1_1')(input_tensor)
    conv1_2 = Conv2D(filters = 1024, kernel_size = (1, 1), strides = (1, 1),
                   activation='relu', padding='same', name='conv1_2')(conv1_1)
    mp1 = MaxPooling2D(pool_size=(2, 2), strides=2, padding='same')(conv1_2)
    
    conv2_1 = Conv2D(filters = 2048, kernel_size = (3, 3), strides = (1, 1),
                   activation='relu', padding='same', name='conv2_1')(mp1)
    conv2_2 = Conv2D(filters = 2048, kernel_size = (1, 1), strides = (1, 1),
                   activation='relu', padding='same', name='conv2_2')(conv2_1)
    mp2 = MaxPooling2D(pool_size=(2, 2), strides=2, padding='same')(conv2_2)
    
    conv3_1 = Conv2D(filters = 4096, kernel_size = (3, 3), strides = (1, 1),
                   activation='relu', padding='same', name='conv3_1')(mp2)
    conv3_2 = Conv2D(filters = 2048, kernel_size = (1, 1), strides = (1, 1),
                   activation='relu', padding='same', name='conv3_2')(conv3_1)
    mp3 = MaxPooling2D(pool_size=(2, 2), strides=2, padding='same')(conv3_2)
    
    conv4_1 = Conv2D(filters = 4096, kernel_size = (3, 3), strides = (1, 1),
                   activation='relu', padding='same', name='conv4_1')(mp3)
    conv4_2 = Conv2D(filters = 2048, kernel_size = (1, 1), strides = (1, 1),
                   activation='relu', padding='same', name='conv4_2')(conv4_1)
    mp4 = MaxPooling2D(pool_size=(2, 2), strides=2, padding='same')(conv4_2)
    
#    flatten = Flatten(name='flatten')(conv8)
#    fc1 = Dense(1024, activation='relu', name='fc1')(flatten)
#    gap = GlobalAveragePooling2D()(conv8)
    gmp = GlobalMaxPooling2D()(mp4)
    drop = Dropout(0.5, name='drop_fc1')(gmp)
    predictions = Dense(2, activation='softmax')(drop)

    model = Model(inputs=input_tensor, outputs=predictions)
    model.summary()

    sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)

parallel_model = multi_gpu_model(model, gpus=4)
#parallel_model.compile(optimizer='Adadelta', 
#                       loss='categorical_crossentropy', 
#                       metrics=['accuracy'])
parallel_model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
parallel_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 512)     0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 128, 128, 1024)    4719616   
_________________________________________________________________
conv1_2 (Conv2D)             (None, 128, 128, 1024)    1049600   
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 1024)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 64, 64, 2048)      18876416  
_________________________________________________________________
conv2_2 (Conv2D)             (None, 64, 64, 2048)      4196352   
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 2048)      0         
__________

In [6]:
print("du", features.shape)
print("du", labels.shape)
parallel_model.fit(x=features, y=labels, 
                   batch_size=16, epochs=100, verbose=1, validation_split=0.2)
model.save_weights("diagnosis.h5")

du (1991, 128, 128, 512)
du (1991, 2)
Train on 1592 samples, validate on 399 samples
Epoch 1/100
1592/1592 [==============================] - 206s 129ms/step - loss: 0.5038 - acc: 0.7594 - val_loss: 0.4878 - val_acc: 0.7594
Epoch 2/100
1592/1592 [==============================] - 207s 130ms/step - loss: 0.4861 - acc: 0.7638 - val_loss: 0.4631 - val_acc: 0.7744
Epoch 3/100
1592/1592 [==============================] - 206s 130ms/step - loss: 0.4674 - acc: 0.7770 - val_loss: 0.4400 - val_acc: 0.8020
Epoch 4/100
1592/1592 [==============================] - 206s 130ms/step - loss: 0.4525 - acc: 0.7984 - val_loss: 0.4024 - val_acc: 0.8471
Epoch 5/100
1592/1592 [==============================] - 207s 130ms/step - loss: 0.4270 - acc: 0.8317 - val_loss: 0.4062 - val_acc: 0.8145
Epoch 6/100
1592/1592 [==============================] - 206s 130ms/step - loss: 0.4200 - acc: 0.8254 - val_loss: 0.4123 - val_acc: 0.8371
Epoch 7/100
1592/1592 [==============================] - 206s 130ms/step - loss: 

Epoch 59/100
1592/1592 [==============================] - 213s 134ms/step - loss: 0.1176 - acc: 0.9460 - val_loss: 0.1826 - val_acc: 0.9198
Epoch 60/100
1592/1592 [==============================] - 213s 134ms/step - loss: 0.1125 - acc: 0.9585 - val_loss: 0.4163 - val_acc: 0.8521
Epoch 61/100
1592/1592 [==============================] - 212s 133ms/step - loss: 0.1026 - acc: 0.9673 - val_loss: 0.2237 - val_acc: 0.9098
Epoch 62/100
1592/1592 [==============================] - 212s 133ms/step - loss: 0.1131 - acc: 0.9629 - val_loss: 0.2359 - val_acc: 0.9023
Epoch 63/100
1592/1592 [==============================] - 212s 133ms/step - loss: 0.0969 - acc: 0.9673 - val_loss: 0.2590 - val_acc: 0.8872
Epoch 64/100
1592/1592 [==============================] - 214s 134ms/step - loss: 0.0790 - acc: 0.9661 - val_loss: 0.2944 - val_acc: 0.8947
Epoch 65/100
1592/1592 [==============================] - 213s 134ms/step - loss: 0.0950 - acc: 0.9673 - val_loss: 0.3210 - val_acc: 0.8922
Epoch 66/100
1592/15